In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

## [DSO] Machine Learning Seminar Vol.7
2020-08-xx  
SKUE

In [2]:
Image(url="https://image.yodobashi.com/product/100/000/009/002/935/105/100000009002935105_10204.jpg")

+ STEP1
 + https://ai.stanford.edu/~amaas/data/sentiment/  
 からダウンロードして、ファイルを適当なフォルダに格納する。
 
+ STEP2  
 + そのディレクトリに移動して、ターミナルで  
 tar -zxf aclImdb_v1.tar.gz  
 を実行する。


In [7]:
import tqdm
import pandas as pd
import os

basepath ='../../../../Documents/dataset/aclImdb/'
labels = { 'pos':1, 'neg':0}

df = pd.DataFrame()

for s in  ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in tqdm.tqdm( os.listdir(path) ):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            
df.columns = ['review', 'sentiment']

100%|██████████| 12500/12500 [00:39<00:00, 316.99it/s]


In [8]:
import numpy as np
np.random.seed(0)

df = df.reindex(np.random.permutation(df.index))

In [11]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [12]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0


## BoWモデル

### 単語を特徴ベクトルに変換

In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
                                'The sun is shining',
                                'The weather is sweet',
                                'The sun is shining, the weather is sweet, and one and one is two'
                            ])
bag = count.fit_transform(docs)

In [18]:
bag

<3x9 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [19]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [20]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### TF-IDFを使って単語の関連性を評価

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf = True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


### テキストデータのクレンジング

In [27]:
df.loc[0, 'review']

'My family and I normally do not watch local movies for the simple reason that they are poorly made, they lack the depth, and just not worth our time.<br /><br />The trailer of "Nasaan ka man" caught my attention, my daughter in law\'s and daughter\'s so we took time out to watch it this afternoon. The movie exceeded our expectations. The cinematography was very good, the story beautiful and the acting awesome. Jericho Rosales was really very good, so\'s Claudine Barretto. The fact that I despised Diether Ocampo proves he was effective at his role. I have never been this touched, moved and affected by a local movie before. Imagine a cynic like me dabbing my eyes at the end of the movie? Congratulations to Star Cinema!! Way to go, Jericho and Claudine!!'

In [26]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '') )
    return text

In [28]:
preprocessor(df.loc[0, 'review'])

'my family and i normally do not watch local movies for the simple reason that they are poorly made they lack the depth and just not worth our time the trailer of nasaan ka man caught my attention my daughter in law s and daughter s so we took time out to watch it this afternoon the movie exceeded our expectations the cinematography was very good the story beautiful and the acting awesome jericho rosales was really very good so s claudine barretto the fact that i despised diether ocampo proves he was effective at his role i have never been this touched moved and affected by a local movie before imagine a cynic like me dabbing my eyes at the end of the movie congratulations to star cinema way to go jericho and claudine '

In [29]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :):(:)'

In [30]:
df['review'] = df['review'].apply(preprocessor)

### 文書をトークン化する